# Fundamentación de la API. Clasificación autosFuel.

Esta API de clasificaicón de autos Fuel está diseñada para facilitar el análisis y la selección de vehículos eficientes para flotas de taxis. Permite evaluar y priorizar vehículos según su rendimiento, emisiones y otras métricas clave, optimizando la inversión en vehículos que sean rentables y sostenibles.

In [52]:
import pandas as pd

# Load the uploaded CSV file to analyze its structure and provide an explanation of the columns
file_path = 'transformed_Vehicle Fuel Economy Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows and column names to understand the structure
data.head(), data.columns

C:\Users\jerom\AppData\Local\Temp\ipykernel_6248\1713813801.py:5: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


(   year manufacturer                          model  barrels08  barrelsa08  \
 0  2017          BMW                       M6 Coupe  17.500588         0.0   
 1  2017          BMW                   X3 sDrive28i  12.396250         0.0   
 2  2016     Infiniti                 Q50S Red Sport  13.523182         0.0   
 3  2017         MINI  John Cooper Works Convertible  11.018889         0.0   
 4  2017          Kia                      Sedona SX  14.167143         0.0   
 
    charge240  city08  city08u  citya08  citya08u  ...  mfrcode  c240dscr  \
 0        0.0    15.0     14.0      0.0       0.0  ...      BMX       NaN   
 1        0.0    21.0     21.0      0.0       0.0  ...      BMX       NaN   
 2        0.0    20.0     19.0      0.0       0.0  ...      NSX       NaN   
 3        0.0    24.0     23.0      0.0       0.0  ...      BMX       NaN   
 4        0.0    18.0     18.0      0.0       0.0  ...      KMX       NaN   
 
    charge240b  c240bdscr   createdon  modifiedon  startstop

In [53]:
from fastapi import FastAPI, Query
from typing import List, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastapi.responses import FileResponse
import os

app = FastAPI()

# Cargar el conjunto de datos
data = pd.read_csv('transformed_Vehicle Fuel Economy Data.csv')

# Filtrar los datos para excluir híbridos y eléctricos
data = data[data['fueltype'].str.contains("Electricity") == False]
data = data[data['fueltype2'].isna()]

# Definir categorías de eficiencia
def categorize_efficiency(row):
    mpg = row['comb08']  # Economía de combustible combinada
    if mpg >= 35:
        return "Eficiencia excelente"
    elif mpg >= 25:
        return "Eficiencia buena"
    elif mpg >= 15:
        return "Eficiencia regular"
    else:
        return "Eficiencia mala"

data['efficiency_category'] = data.apply(categorize_efficiency, axis=1)

@app.get("/efficiency_analysis/")
def efficiency_analysis(category: Optional[str] = None):
    categories = ["Eficiencia excelente", "Eficiencia buena", "Eficiencia regular", "Eficiencia mala"]

    if category and category not in categories:
        return {"error": f"Categoría inválida. Las categorías válidas son: {categories}"}

    selected_categories = [category] if category else categories

    all_results = []

    for category in selected_categories:
        filtered_data = data[data['efficiency_category'] == category]

        if filtered_data.empty:
            continue

        # Seleccionar las columnas numéricas relevantes
        numeric_columns = ['range', 'city08', 'highway08', 'co2', 'ghgscore', 'comb08']
        analysis = filtered_data[[
            'manufacturer', 'model', 'year', 'vclass', 'trany', 'fueltype', *numeric_columns
        ]]

        # Ordenar todos los modelos por economía de combustible combinada
        sorted_data = analysis.sort_values(by='comb08', ascending=False).reset_index(drop=True)

        # Agregar los resultados al conjunto global
        sorted_data['efficiency_category'] = category
        all_results.append(sorted_data)

    # Combinar todos los resultados
    final_data = pd.concat(all_results, ignore_index=True)

    # Exportar a CSV
    output_path = "efficiency_analysis.csv"
    final_data.to_csv(output_path, index=False)

    return FileResponse(output_path, media_type="text/csv", filename="efficiency_analysis.csv")

@app.get("/efficiency_graphs/")
def efficiency_graphs(category: Optional[str] = None):
    categories = ["Eficiencia excelente", "Eficiencia buena", "Eficiencia regular", "Eficiencia mala"]

    if category and category not in categories:
        return {"error": f"Categoría inválida. Las categorías válidas son: {categories}"}

    selected_categories = [category] if category else categories
    output_dir = "graphs"
    os.makedirs(output_dir, exist_ok=True)

    for category in selected_categories:
        filtered_data = data[data['efficiency_category'] == category]

        if filtered_data.empty:
            continue

        # Seleccionar las columnas numéricas relevantes
        numeric_columns = ['range', 'city08', 'highway08', 'co2', 'ghgscore', 'comb08']
        analysis = filtered_data[[
            'manufacturer', 'model', 'year', 'vclass', 'trany', 'fueltype', *numeric_columns
        ]]

        # Obtener los 15 mejores modelos por economía de combustible combinada
        top_15 = analysis.nlargest(15, 'comb08')

        # Crear un gráfico separado para cada métrica numérica
        for column in numeric_columns:
            plt.figure(figsize=(12, 8))
            x = np.arange(len(top_15))
            y = top_15[column].values

            # Calcular la media
            y_mean = np.mean(y)

            # Gráfico con datos reales y línea de la media
            plt.bar(top_15['model'], y, color='skyblue', label='Datos reales')
            plt.axhline(y=y_mean, color='red', linestyle='--', label=f'Media: {y_mean:.2f}')

            plt.title(f"Top 15 modelos por {column} - {category}")
            plt.xlabel("Modelos")
            plt.ylabel(column)
            plt.xticks(rotation=45, ha='right')
            plt.legend()
            plt.tight_layout()

            # Guardar el gráfico
            graph_path = os.path.join(output_dir, f"{category.replace(' ', '_')}_{column}.png")
            plt.savefig(graph_path)
            plt.close()

    return FileResponse(output_dir, media_type="application/zip", filename="efficiency_graphs.zip")


C:\Users\jerom\AppData\Local\Temp\ipykernel_6248\3135468582.py:12: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('transformed_Vehicle Fuel Economy Data.csv')


In [54]:
efficiency_analysis("Eficiencia buena")

In [55]:
efficiency_graphs ("Eficiencia buena")

In [56]:
efficiency_graphs ("Eficiencia excelente")

# Flujo de Trabajo
1. Identificación de Necesidades:
    - La agencia determina qué métricas son importantes para su operación (e.g., rendimiento de combustible, emisiones bajas, costos de operación).
    - Las categorías predefinidas ("Eficiencia excelente", "Eficiencia buena", etc.) permiten organizar vehículos según su idoneidad para el uso intensivo como taxis.
2. Carga y Filtrado de Datos:
    - La API utiliza un conjunto de datos de vehículos que incluye información sobre consumo, emisiones y rango.
    - Se excluyen vehículos eléctricos e híbridos para centrarse en opciones convencionales.
3. Análisis de Eficiencia:
    - La API clasifica los vehículos en cuatro categorías de eficiencia basadas en su consumo combinado (millas por galón, MPG).
    - Los datos analizados incluyen:
        + Rango (capacidad de operar largas distancias entre recargas/repostajes).
        + Consumo en ciudad y carretera (ideal para diferentes condiciones de tráfico).
        + Emisiones de CO2 (para cumplir con regulaciones ambientales).
4. Generación de Reportes:

    - Exportación CSV:
        + Un archivo CSV que lista todos los vehículos clasificados por categoría y ordenados de mayor a menor eficiencia.
        + Incluye detalles como fabricante, modelo, año, rango y emisiones.
    - Gráficos Comparativos:
        + Gráficos visuales para cada métrica clave (e.g., rango, consumo en ciudad) que permiten comparar rápidamente los vehículos más destacados.
5. Toma de Decisiones:
    - La agencia utiliza los datos generados para:
        + Seleccionar vehículos con mayor rendimiento y menor impacto ambiental.
        + Priorizar vehículos que optimicen costos operativos (combustible y mantenimiento).

# Beneficios para la Agencia de Transporte

1. Optimización de Costos:
    - Selección de vehículos con mejor rendimiento en consumo de combustible.
    - Identificación de opciones que minimizan costos operativos.
2. Cumplimiento Ambiental: Elección de vehículos con menores emisiones de CO2 para cumplir con regulaciones y mejorar la percepción pública.
3. Facilitación de la Toma de Decisiones: Los reportes detallados y gráficos visuales ofrecen una visión clara de las opciones disponibles.
4. Escalabilidad: La API puede analizar grandes volúmenes de datos para futuras expansiones de flota o nuevas adquisiciones.

# Posibles mejoras de la API. 
Las mejoras incluyen aplicación de machine learning. 

1. Predicción de Costos Operativos por modelo
    - Idea: Usar Machine Learning para predecir costos anuales basados en consumo, rango y emisiones.
    - Beneficio: Ayuda a elegir vehículos más rentables a largo plazo.
2. Clustering de Vehículos por Uso
    - Idea: Agrupar vehículos según características como rango y consumo (e.g., "óptimos para ciudad", "óptimos para largas distancias").
    - Beneficio: Optimiza el uso de la flota según las condiciones operativas.
3. Recomendación Basada en Prioridades
    - Idea: Crear un sistema de recomendación que clasifique vehículos según prioridades definidas (e.g., eficiencia, emisiones).
    - Beneficio: Permite personalizar la selección de vehículos según necesidades del negocio.

# Aplicación de mejora 1. Ejemplo



In [57]:
from fastapi import FastAPI, Query
from typing import List, Optional
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from fastapi.responses import FileResponse
import os

app = FastAPI()

# Cargar el conjunto de datos
data = pd.read_csv('transformed_Vehicle Fuel Economy Data.csv')

# Filtrar los datos para excluir híbridos y eléctricos
data = data[data['fueltype'].str.contains("Electricity") == False]
data = data[data['fueltype2'].isna()]

# Asegurarse de que las columnas numéricas sean realmente numéricas
numeric_columns = ['range', 'city08', 'highway08', 'co2', 'ghgscore', 'comb08']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Eliminar filas con valores nulos en las columnas numéricas relevantes
data = data.dropna(subset=numeric_columns)

# Definir categorías de eficiencia
def categorize_efficiency(row):
    mpg = row['comb08']  # Economía de combustible combinada
    if mpg >= 35:
        return "Eficiencia excelente"
    elif mpg >= 25:
        return "Eficiencia buena"
    elif mpg >= 15:
        return "Eficiencia regular"
    else:
        return "Eficiencia mala"

data['efficiency_category'] = data.apply(categorize_efficiency, axis=1)

@app.get("/efficiency_analysis/")
def efficiency_analysis(category: Optional[str] = None):
    categories = ["Eficiencia excelente", "Eficiencia buena", "Eficiencia regular", "Eficiencia mala"]

    if category and category not in categories:
        return {"error": f"Categoría inválida. Las categorías válidas son: {categories}"}

    selected_categories = [category] if category else categories

    all_results = []

    for category in selected_categories:
        filtered_data = data[data['efficiency_category'] == category]

        if filtered_data.empty:
            continue

        # Seleccionar las columnas numéricas relevantes
        analysis = filtered_data[[
            'manufacturer', 'model', 'year', 'vclass', 'trany', 'fueltype', *numeric_columns
        ]]

        # Ordenar todos los modelos por economía de combustible combinada
        sorted_data = analysis.sort_values(by='comb08', ascending=False).reset_index(drop=True)

        # Agregar los resultados al conjunto global
        sorted_data['efficiency_category'] = category
        all_results.append(sorted_data)

    # Combinar todos los resultados
    final_data = pd.concat(all_results, ignore_index=True)

    # Exportar a CSV
    output_path = "efficiency_analysis.csv"
    final_data.to_csv(output_path, index=False)

    return FileResponse(output_path, media_type="text/csv", filename="efficiency_analysis.csv")

@app.post("/predict_operating_cost_by_model/")
def predict_operating_cost_by_model(model_name: str):
    # Filtrar el modelo específico
    model_data = data[data['model'].str.contains(model_name, case=False, na=False)]

    if model_data.empty:
        return {"error": f"No se encontró información para el modelo '{model_name}'"}

    # Seleccionar columnas relevantes para la predicción
    model_features = model_data[['city08', 'highway08', 'range', 'co2']].mean()

    # Crear un modelo de regresión lineal ficticio con datos simulados
    training_data = {
        "city08": np.random.uniform(10, 40, 100),
        "highway08": np.random.uniform(15, 50, 100),
        "range": np.random.uniform(200, 600, 100),
        "co2": np.random.uniform(100, 400, 100),
        "cost": np.random.uniform(2000, 8000, 100),
    }
    training_df = pd.DataFrame(training_data)

    X = training_df[["city08", "highway08", "range", "co2"]]
    y = training_df["cost"]

    model = LinearRegression()
    model.fit(X, y)

    # Predecir el costo basado en los valores promedio del modelo
    input_data = model_features.values.reshape(1, -1)
    predicted_cost = model.predict(input_data)

    return {
        "Predicted Operating Cost": f"${predicted_cost[0]:.2f} por año",
        "Model": model_name,
        "Inputs (Promedios del Modelo)": {
            "city08 (MPG en ciudad)": model_features['city08'],
            "highway08 (MPG en carretera)": model_features['highway08'],
            "range (millas)": model_features['range'],
            "co2 (g/milla)": model_features['co2'],
        },
    }


C:\Users\jerom\AppData\Local\Temp\ipykernel_6248\3983274898.py:12: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('transformed_Vehicle Fuel Economy Data.csv')


In [58]:
efficiency_analysis("Eficiencia excelente")

In [85]:
predict_operating_cost_by_model("Ioniq Blue")

c:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


{'Predicted Operating Cost': '$5849.80 por año',
 'Model': 'Ioniq Blue',
 'Inputs (Promedios del Modelo)': {'city08 (MPG en ciudad)': 57.333333333333336,
  'highway08 (MPG en carretera)': 59.333333333333336,
  'range (millas)': 0.0,
  'co2 (g/milla)': 153.0}}

In [88]:
predict_operating_cost_by_model("Accord Hybrid")

c:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


{'Predicted Operating Cost': '$3601.95 por año',
 'Model': 'Accord Hybrid',
 'Inputs (Promedios del Modelo)': {'city08 (MPG en ciudad)': 42.8,
  'highway08 (MPG en carretera)': 42.46666666666667,
  'range (millas)': 0.0,
  'co2 (g/milla)': 152.0}}

# Explicación de la API Aplicada al Negocio
Esta API está diseñada para apoyar a una agencia de transporte o empresa interesada en optimizar su flota de taxis. Proporciona herramientas para analizar la eficiencia de los vehículos y predecir costos operativos, lo que permite tomar decisiones informadas al adquirir o gestionar vehículos. Vale aclarar que el algoritmo esta trabajando con datos ficticios, en la medida que avance el proyecto se puede trabajar con datos reales. 

# Flujo y Aplicación de los Endpoints

1. Análisis de Eficiencia de Vehículos (/efficiency_analysis/)
    - ¿Qué hace?
        + Clasifica los vehículos en categorías como "Eficiencia excelente", "Eficiencia buena", etc.
        + Genera un archivo CSV con información detallada de los vehículos, ordenados por su eficiencia combinada.
    - Aplicación para el negocio:
        + Permite identificar rápidamente los vehículos más eficientes para el uso en flotas de taxis, reduciendo costos de combustible y emisiones.
        + Ayuda a priorizar modelos según su rendimiento en ciudad y carretera, ideal para diferentes contextos operativos.
2. Predicción de Costo Operativo por Modelo (/predict_operating_cost_by_model/)
    - ¿Qué hace?
        + Permite ingresar el nombre de un modelo de auto.
        + La API filtra los datos históricos del modelo y utiliza sus características promedio (city08, highway08, range, co2) para predecir el costo operativo anual usando un modelo de regresión.
    - Aplicación para el negocio:
        + Proporciona una estimación clara del costo operativo para cada modelo, basada en datos de rendimiento y emisiones.
        + Ayuda a calcular el retorno de inversión (ROI) de vehículos específicos al considerar costos a largo plazo.
        + Reduce riesgos al elegir vehículos que resulten costosos de operar.

# Impacto en el Negocio
1. Optimización de Inversiones: La agencia puede enfocar su presupuesto en vehículos con un equilibrio ideal entre eficiencia y costo operativo, maximizando la rentabilidad de su flota.
2. Toma de Decisiones Informada: Los datos generados permiten comparar modelos en función de métricas clave como emisiones y consumo, facilitando la selección de vehículos que cumplan con regulaciones ambientales y necesidades operativas.
3. Alineación con Sostenibilidad: Promueve la elección de vehículos con bajas emisiones de CO2, mejorando la percepción de la marca y cumpliendo objetivos de responsabilidad ambiental.